# Re-ranking

Amazon Bedrock provides access to reranker models that you can use when querying to improve the relevance of the retrieved results.  reranker model calculates the relevance of chunks to a query and reorders the results based on the scores that it calculates. By using a reranker model, you can return responses that are better suited to answering the query. 

Reranker models are trained to identify relevance signals based on a query and then use those signals to rank documents. Because of this, the models can provide more relevant, more accurate results.

If you're using `Amazon Bedrock Knowledge Bases` for building your Retrieval Augmented Generation (RAG) application, use a reranker model while calling the `Retrieve` or `RetrieveAndGenerate operation`. The results from reranking override the default ranking that Amazon Bedrock Knowledge Bases determines.

This notebook demonstrates the use of **reranking model** with Amazon Bedrock Knowledge Bases, through the Rerank API which will help to further improve the accuracy and relevance of RAG applications. With a reranker model, you can retrieve fewer, but more relevant, results. By feeding these results to the foundation model that you use to generate a response, you can also decrease cost and latency.

Let's explore how to implement and utilize reranking models with Amazon Bedrock Knowledge Bases for an example use case.

## 1. Setup
Before running the rest of this notebook, you'll need to run the cells below to (ensure necessary libraries are installed and) connect to Bedrock.

Please ignore any pip dependency error (if you see any while installing libraries)

In [1]:
%pip install --force-reinstall -q -r utils/requirements.txt --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install --upgrade boto3

Note: you may need to restart the kernel to use updated packages.


In [3]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [1]:
import boto3
print(boto3.__version__)

1.35.79


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
import time
import boto3
import logging
import pprint
import json

from utils.knowledge_base import BedrockKnowledgeBase

In [4]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session(region_name = 'us-west-2')
region =  session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime') 
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

('us-west-2', '533267284022')

In [5]:
import time

# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"
knowledge_base_name = 'reranking-kb'
knowledge_base_description = "Knowledge Base for re-ranking."
bucket_name = f'{knowledge_base_name}-{suffix}'
foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

## 2 - Create knowledge bases with fixed chunking strategy
Let's start by creating a [Knowledge Base for Amazon Bedrock](https://aws.amazon.com/bedrock/knowledge-bases/) to store video games data in csv format. Knowledge Bases allow you to integrate with different vector databases including [Amazon OpenSearch Serverless](https://aws.amazon.com/opensearch-service/features/serverless/), [Amazon Aurora](https://aws.amazon.com/rds/aurora/), [Pinecone](http://app.pinecone.io/bedrock-integration), [Redis Enterprise]() and [MongoDB Atlas](). For this example, we will integrate the knowledge base with Amazon OpenSearch Serverless. To do so, we will use the helper class `BedrockKnowledgeBase` which will create the knowledge base and all of its pre-requisites:
1. IAM roles and policies
2. S3 bucket
3. Amazon OpenSearch Serverless encryption, network and data access policies
4. Amazon OpenSearch Serverless collection
5. Amazon OpenSearch Serverless vector index
6. Knowledge base
7. Knowledge base data source

We will create a knowledge base using fixed chunking strategy. 

You can chhose different chunking strategies by changing the below parameter values: 
```
"chunkingStrategy": "FIXED_SIZE | NONE | HIERARCHICAL | SEMANTIC"
```

In [6]:
knowledge_base_metadata = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name}-{suffix}',
    kb_description=knowledge_base_description,
    data_bucket_name=bucket_name, 
    chunking_strategy = "FIXED_SIZE", 
    suffix = suffix
)

[2024-12-12 10:11:26,384] p71543 {credentials.py:1278} INFO - Found credentials in shared credentials file: ~/.aws/credentials
[2024-12-12 10:11:27,443] p71543 {credentials.py:1278} INFO - Found credentials in shared credentials file: ~/.aws/credentials



 Region:  us-east-1
Step 1 - Creating or retrieving S3 bucket(s) for Knowledge Base documents
['reranking-kb-2090331']
Creating bucket reranking-kb-2090331
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_2090331) and Policies
Step 3 - Creating OSS encryption, network and data access policies
Step 4 - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '318',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Thu, 12 Dec 2024 18:11:30 '
                                                 'GMT',
                                         'x-amzn-requestid': '8e1f42c3-5067-443b-8bb3-c39387073753'},
                        'HTTPStatusCode': 200,
                        'RequestId': '8e1f42c3-5067-443b-8bb3-c39387073753

[2024-12-12 10:13:02,007] p71543 {base.py:258} INFO - PUT https://1ldzs4drtk7hzo39kj5e.us-east-1.aoss.amazonaws.com:443/bedrock-sample-rag-index-2090331 [status:200 request:0.721s]



Creating index:
{ 'acknowledged': True,
  'index': 'bedrock-sample-rag-index-2090331',
  'shards_acknowledged': True}
Step 6 - Will create Lambda Function if chunking strategy selected as CUSTOM
Not creating lambda function as chunking strategy is FIXED_SIZE
Step 7 - Creating Knowledge Base
Creating KB with chunking strategy - FIXED_SIZE
============Chunking config========
 {'chunkingConfiguration': {'chunkingStrategy': 'FIXED_SIZE', 'fixedSizeChunkingConfiguration': {'maxTokens': 300, 'overlapPercentage': 20}}}
{ 'createdAt': datetime.datetime(2024, 12, 12, 18, 14, 2, 726331, tzinfo=tzutc()),
  'description': 'Knowledge Base for re-ranking.',
  'knowledgeBaseArn': 'arn:aws:bedrock:us-east-1:533267284022:knowledge-base/VWQGYOZQLB',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-embed-text-v2:0'}},
  'knowledgeBaseId': 'VWQGYOZQLB',

### 2.1 Download Amazon 2019, 2020, 2021, 2022, & 2023 annual reports and upload it to Amazon S3

Now that we have created the knowledge base, let's populate it with the `sec-10-k reports` dataset to KB. This data is being downloaded from [here](https://ir.aboutamazon.com/annual-reports-proxies-and-shareholder-letters/default.aspx). This data is about Amazon's annual reports, proxies and shareholder letters.

In [ ]:
import os

def create_directory(directory_name):    
    if not os.path.exists(directory_name):
        os.makedirs(directory_name)
        print(f"Directory '{directory_name}' created successfully.")
    else:
        print(f"Directory '{directory_name}' already exists.")

# Call the function to create the directory
create_directory("sec-10-k")

In [7]:
import requests

def download_file(url, filename):
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Open the file in write-binary mode
        with open(filename, 'wb') as file:
            # Write the content of the response to the file
            file.write(response.content)
        print(f"File downloaded successfully: {filename}")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")

# URL of the files to download
urls = ["https://s2.q4cdn.com/299287126/files/doc_financials/2024/ar/Amazon-com-Inc-2023-Annual-Report.pdf",
        "https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/Amazon-2022-Annual-Report.pdf",
        "https://s2.q4cdn.com/299287126/files/doc_financials/2022/ar/Amazon-2021-Annual-Report.pdf",
        "https://s2.q4cdn.com/299287126/files/doc_financials/2021/ar/Amazon-2020-Annual-Report.pdf",
        "https://s2.q4cdn.com/299287126/files/doc_financials/2020/ar/2019-Annual-Report.pdf"]


for url in urls:
    # Name for the downloaded file
    filename = url.split('/')[-1]

    # Path to save the downloaded file
    filepath = f"./sec-10-k/{filename}"

    # Call the function to download the file
    download_file(url, filepath)

File downloaded successfully: ./sec-10-k/Amazon-com-Inc-2023-Annual-Report.pdf
File downloaded successfully: ./sec-10-k/Amazon-2022-Annual-Report.pdf
File downloaded successfully: ./sec-10-k/Amazon-2021-Annual-Report.pdf
File downloaded successfully: ./sec-10-k/Amazon-2020-Annual-Report.pdf
File downloaded successfully: ./sec-10-k/2019-Annual-Report.pdf


Let's upload the annual reports data available in the `sec-10-k` folder to s3.

In [8]:
def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                if not file.startswith('.DS_Store'):
                    file_to_upload = os.path.join(root,file)
                    print(f"uploading file {file_to_upload} to {bucket_name}")
                    s3_client.upload_file(file_to_upload,bucket_name,file)

# upload metadata file to S3
upload_directory("sec-10-k", bucket_name)

uploading file sec-10-k/Amazon-2022-Annual-Report.pdf to reranking-kb-2090331
uploading file sec-10-k/2019-Annual-Report.pdf to reranking-kb-2090331
uploading file sec-10-k/Amazon-2020-Annual-Report.pdf to reranking-kb-2090331
uploading file sec-10-k/Amazon-com-Inc-2023-Annual-Report.pdf to reranking-kb-2090331
uploading file sec-10-k/Amazon-2021-Annual-Report.pdf to reranking-kb-2090331


Now start the ingestion job. Since, we are using the same documents as used for fixed chunking, we are skipping the step to upload documents to s3 bucket. 

In [9]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base_metadata.start_ingestion_job()

{ 'dataSourceId': 'Y7CZBO3FI6',
  'ingestionJobId': '2R7LRHDIHW',
  'knowledgeBaseId': 'VWQGYOZQLB',
  'startedAt': datetime.datetime(2024, 12, 12, 18, 20, 52, 918706, tzinfo=tzutc()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 0,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 0},
  'status': 'STARTING',
  'updatedAt': datetime.datetime(2024, 12, 12, 18, 20, 52, 918706, tzinfo=tzutc())}
{ 'dataSourceId': 'Y7CZBO3FI6',
  'ingestionJobId': '2R7LRHDIHW',
  'knowledgeBaseId': 'VWQGYOZQLB',
  'startedAt': datetime.datetime(2024, 12, 12, 18, 20, 52, 918706, tzinfo=tzutc()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 5,
     

Finally we save the Knowledge Base Id to test the solution at a later stage. 

In [10]:
kb_id = knowledge_base_metadata.get_knowledge_base_id()

'VWQGYOZQLB'


## 3. Evaluate the relevance of query responses with and without Re-ranking (using Ragas)

Define models for generation, evaluation and re-ranking

In [11]:
from langchain.llms.bedrock import Bedrock
from langchain_aws import ChatBedrock
from langchain_aws import BedrockEmbeddings

bedrock_client = boto3.client('bedrock-runtime')

TEXT_GENERATION_MODEL_ID = "anthropic.claude-3-haiku-20240307-v1:0"
EVALUATION_MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0"
EMBEDDING_MODEL_ID = "amazon.titan-embed-text-v2:0"

# Reranker model: there are two reranker models available at launch
AMAZON_RERANKER_MODEL_ID = "amazon.rerank-v1:0"
COHERE_RERANKER_MODEL_ID = "cohere.rerank-v3-5:0"


llm_for_evaluation = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0", client=bedrock_client)
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0", client=bedrock_client)


#### 3.1 Update Knowledge Bases execution role

In [12]:
# Before using autogenerated filters - update the knowledge base execution IAM role with right permissions

iam = boto3.resource('iam')
client = boto3.client('iam')

def get_attached_policies(role_name):
    response = client.list_attached_role_policies(RoleName=role_name)
    attached_policies = response['AttachedPolicies']
    return attached_policies

# get the knowledge base IAM role name
get_kb_response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId = kb_id)
role_arn = get_kb_response['knowledgeBase']['roleArn']
role_name = role_arn.split('/')[-1]

# get attached policies
attached_policies = get_attached_policies(role_name)
attached_policies

def update_kb_execution_role(attached_policies, region_name):
    
    for policy in attached_policies:

        print(policy['PolicyArn'])
        policy_name = policy['PolicyName']
        policy_arn = policy['PolicyArn']

        if 'FoundationModel' in policy_arn:
            print('Updating FoundationModel policy: ',policy_arn)
            policy = iam.Policy(policy_arn)
            version = policy.default_version
            policyJson = version.document
            policyJson['Statement'][0]['Resource'].append(f'arn:aws:bedrock:{region}::foundation-model/{TEXT_GENERATION_MODEL_ID}')
            policyJson['Statement'][0]['Resource'].append(f'arn:aws:bedrock:{region}::foundation-model/{EVALUATION_MODEL_ID}')  
            policyJson['Statement'][0]['Resource'].append(f'arn:aws:bedrock:{region}::foundation-model/{AMAZON_RERANKER_MODEL_ID}') 
            policyJson['Statement'][0]['Resource'].append(f'arn:aws:bedrock:{region}::foundation-model/{COHERE_RERANKER_MODEL_ID}') 
        
            client.detach_role_policy(RoleName=role_name,
                PolicyArn=policy_arn)
            
            response = client.delete_policy(
                PolicyArn=policy_arn
            )
            print(response)
           
            response = client.create_policy(
            PolicyName= policy_name,
            PolicyDocument=json.dumps(policyJson)
            )
            print(response)
        
        client.attach_role_policy(
            RoleName=role_name,
            PolicyArn=policy_arn
        )

In [13]:
update_kb_execution_role(attached_policies, region)
# time.sleep(30)

arn:aws:iam::533267284022:policy/AmazonBedrockCloudWatchPolicyForKnowledgeBase_2090331
arn:aws:iam::533267284022:policy/AmazonBedrockOSSPolicyForKnowledgeBase_2090331
arn:aws:iam::533267284022:policy/AmazonBedrockS3PolicyForKnowledgeBase_2090331
arn:aws:iam::533267284022:policy/AmazonBedrockFoundationModelPolicyForKnowledgeBase_2090331
Updating FoundationModel policy:  arn:aws:iam::533267284022:policy/AmazonBedrockFoundationModelPolicyForKnowledgeBase_2090331
{'ResponseMetadata': {'RequestId': '006f85fc-1c5d-420d-8add-e93c6f3a68d5', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 12 Dec 2024 18:25:20 GMT', 'x-amzn-requestid': '006f85fc-1c5d-420d-8add-e93c6f3a68d5', 'content-type': 'text/xml', 'content-length': '204'}, 'RetryAttempts': 0}}
{'Policy': {'PolicyName': 'AmazonBedrockFoundationModelPolicyForKnowledgeBase_2090331', 'PolicyId': 'ANPAXYKJVAA3E2WGZSAJA', 'Arn': 'arn:aws:iam::533267284022:policy/AmazonBedrockFoundationModelPolicyForKnowledgeBase_2090331', 'Path': '/', 'Defau

#### 3.2 Customize retrieve and generate configuraion

In [14]:
def retrieve_and_generate(query, reranker_model=None, kb_id=None, TEXT_GENERATION_MODEL_ID=None, metadata_filters=None):
    
    # Prepare retrieval configuration
    retrieval_config = {
        "vectorSearchConfiguration": {
            "numberOfResults": 30 if reranker_model else 3
        }
    }

    if reranker_model:
        retrieval_config["vectorSearchConfiguration"]["rerankingConfiguration"] = {
            "type": "BEDROCK_RERANKING_MODEL",
            "bedrockRerankingConfiguration": {
                "modelConfiguration": {
                    "modelArn": f'arn:aws:bedrock:{region}::foundation-model/{reranker_model}',
                },
                "numberOfRerankedResults": 3
            }
        }

        if metadata_filters:
            retrieval_config["vectorSearchConfiguration"]["rerankingConfiguration"]["bedrockRerankingConfiguration"]["metadataConfiguration"] = {
                                                                "selectionMode" : "SELECTIVE",
                                                                "selectiveModeConfiguration" : {
                                                                    "fieldsToInclude": [{
                                                                        "fieldName": "year",
                                                                    }]
                                                                }
                                                            }
                    

    # Call the retrieve and generate API
    start = time.time()
    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={'text': query},
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': kb_id,
                'modelArn': f'arn:aws:bedrock:{region}::foundation-model/{TEXT_GENERATION_MODEL_ID}',
                'retrievalConfiguration': retrieval_config,
            },
        }
    )
    time_spent = time.time() - start

    print(f"[Response] : {response['output']['text']}\n")
    print(f"[Invocation time] : {time_spent}\n")

    return response


#### 3.3 Prepare dataset for evaluation

In [15]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    context_relevancy,
    context_recall,
    context_precision,
    context_entity_recall,
    answer_correctness,
)

#specify the metrics here
metrics = [
    context_relevancy,
    context_recall,
    context_precision,
    context_entity_recall,
    answer_correctness,
]

questions = [
    "How many days has Amazon asked employees to come to work in office in 2022?",
    "By what percentage did AWS revenue grow year-over-year in 2022?",
    "Compared to Graviton2 processors, what performance improvement did Graviton3 chips deliver according to the passage?",
    "Which was the first inference chip launched by AWS according to the passage?",
    "According to the context, in what year did Amazon's annual revenue increase from $245B to $434B?"
]
ground_truths = [
    "Amazon has asked corporate employees to come back to office at least three days a week beginning May 2022.",
    "AWS had a 29% year-over-year ('YoY') revenue in 2022 on $62B revenue base.",
    "In 2022, AWS delivered their Graviton3 chips, providing 25% better performance than the Graviton2 processors.",
    "AWS launched their first inference chips (“Inferentia”) in 2019, and they have saved companies like Amazon over a hundred million dollars in capital expense.",
    "Amazon's annual revenue increased from $245B in 2019 to $434B in 2022."

]

[2024-12-12 10:25:42,474] p71543 {config.py:54} INFO - PyTorch version 2.4.0 available.
/opt/homebrew/Caskroom/miniforge/base/envs/aws_p3.10/lib/python3.10/site-packages/ragas/metrics/__init__.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from ragas.metrics._answer_correctness import AnswerCorrectness, answer_correctness
/opt/homebrew/Caskroom/miniforge/base/envs/aws_p3.10/lib/python3.10/site-packages/ragas/metrics/__init__.py:4: LangChainDeprecationWarning: As of langchain-core 0.3.0, La

In [16]:
def prepare_eval_dataset(questions, ground_truths, kb_id=None, TEXT_GENERATION_MODEL_ID=None, reranker_model=None, metadata_filters = None):
    answers = []
    contexts = []
    
    for query in questions:
        response = retrieve_and_generate(
            query,
            reranker_model=reranker_model,
            kb_id=kb_id,
            TEXT_GENERATION_MODEL_ID=TEXT_GENERATION_MODEL_ID,
            metadata_filters=metadata_filters
        )
        
        answers.append(response["output"]["text"])
        
        context_group = []
        for citation in response["citations"]:
            context_group.extend([
                ref["content"]["text"]
                for ref in citation["retrievedReferences"]
                if "content" in ref and "text" in ref["content"]
            ])
        contexts.append(context_group)
        time.sleep(15)

    # Create dictionary
    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truth": ground_truths
    }

    # Convert dict to dataset
    dataset = Dataset.from_dict(data)
    return dataset


#### 3.4 Evaluate dataset - without re-ranker

In [17]:
without_reranker_dataset = prepare_eval_dataset(questions, ground_truths, kb_id, TEXT_GENERATION_MODEL_ID, reranker_model=None)

ValidationException: An error occurred (ValidationException) when calling the RetrieveAndGenerate operation: The provided ARN is invalid for the service region (Service: BedrockRuntime, Status Code: 400, Request ID: bf1f127e-6415-4320-8dc5-b9bea9ce7959)

In [ ]:
without_reranker_result = evaluate(
    dataset=without_reranker_dataset,
    metrics=metrics,
    llm=llm_for_evaluation,
    embeddings=bedrock_embeddings,
)

without_reranker_result_df = without_reranker_result.to_pandas()

#### 3.5 Evaluate dataset - with re-ranker

In [ ]:
with_reranker_dataset = prepare_eval_dataset(questions, ground_truths, kb_id, TEXT_GENERATION_MODEL_ID, reranker_model=AMAZON_RERANKER_MODEL_ID)

In [ ]:
with_reranker_result = evaluate(
dataset=with_reranker_dataset,
metrics=metrics,
llm=llm_for_evaluation,
embeddings=bedrock_embeddings,
)

with_reranker_result_df = with_reranker_result.to_pandas()

#### 3.4 Evaluate dataset - with re-ranker + metadata configuration

##### 3.4.1 Prepare metadata for ingestion


In [25]:
import json
import re

def generate_matadata(data_dir):
    
    # Loop through all PDF files in the directory
    for filename in os.listdir(data_dir):
        if not filename.startswith('.DS_Store'):
            # Define the metadata dictionary
            metadata ={}
            
            filename= f'{data_dir}/{filename}'
            print(filename)
            
            # Create metadata
            metadata["company"] = "Amazon"
            metadata["ticker"] = "AMZN"
            metadata["year"] = re.search(r'\d+', filename.split('/')[-1]).group(0)

            # Create a JSON object
            json_data = {"metadataAttributes": metadata}

            # print(json_data)

            # Write the JSON object to a file
            with open(f"{filename.replace('.pdf', '.pdf.metadata.json')}", "w") as f:
                json.dump(json_data, f)


In [ ]:
data_dir = './sec-10-k'
generate_matadata(data_dir)

In [ ]:
# upload metadata file to S3
upload_directory("sec-10-k", bucket_name)

##### 3.4.2 Ingest metadata into Knowledge Bases


Now start the ingestion job. Since, we are using the same documents as used for fixed chunking, we are skipping the step to upload documents to s3 bucket. 

In [ ]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base_metadata.start_ingestion_job()

In [ ]:
with_reranker_metadata_filters_dataset = prepare_eval_dataset(questions, ground_truths, kb_id, TEXT_GENERATION_MODEL_ID, reranker_model=AMAZON_RERANKER_MODEL_ID, metadata_filters=True)

In [ ]:
with_reranker_metadata_filters_result = evaluate(
dataset=with_reranker_metadata_filters_dataset,
metrics=metrics,
llm=llm_for_evaluation,
embeddings=bedrock_embeddings,
)

with_reranker_metadata_filters_result_df = with_reranker_result.to_pandas()

#### 3.5 Prepare Comparison data frame

In [31]:
import pandas as pd

# Create the side-by-side DataFrame
comparison_df = pd.DataFrame({
    'question': without_reranker_result_df['question'],
    'without_reranker_answer': without_reranker_result_df['answer'],
    'with_reranker_answer': with_reranker_result_df['answer'],
    'with_reranker_metadata_answer': with_reranker_metadata_filters_result_df['answer'],
    
    'without_reranker_answer_correctness': without_reranker_result_df['answer_correctness'],
    'with_reranker_answer_correctness': with_reranker_result_df['answer_correctness'],
    'with_reranker_metadata_correctness': with_reranker_metadata_filters_result_df['answer_correctness'],
    })

In [ ]:
pd.options.display.max_colwidth = 1000
comparison_df

In [ ]:
# Calculate average correctness
without_reranker_avg_correctness = without_reranker_result_df['answer_correctness'].mean()
with_reranker_avg_correctness = with_reranker_result_df['answer_correctness'].mean()
with_reranker_metadata_avg_correctness = with_reranker_metadata_filters_result_df['answer_correctness'].mean()

print(f"\nAverage Correctness without Reranker: {without_reranker_avg_correctness:.4f}")
print(f"Average Correctness with Reranker: {with_reranker_avg_correctness:.4f}")
print(f"Average Correctness with Reranker and metadata filter: {with_reranker_metadata_avg_correctness:.4f}")

### 2.7 Clean up
Please make sure to uncomment and run below cells to delete the resources created in this notebook.

In [18]:
# delete local directory
import shutil

dir_path = "sec-10-k" # Replace with the actual path

try:
    shutil.rmtree(dir_path)
    print(f"Directory '{dir_path}' and its contents have been deleted successfully.")
except FileNotFoundError:
    print(f"Directory '{dir_path}' not found.")
except Exception as e:
        print(f"An error occurred: {e}")

Directory 'sec-10-k' and its contents have been deleted successfully.


In [19]:
## Empty and delete S3 Bucket

objects = s3_client.list_objects(Bucket=bucket_name)  
if 'Contents' in objects:
    for obj in objects['Contents']:
        s3_client.delete_object(Bucket=bucket_name, Key=obj['Key']) 
s3_client.delete_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': 'DKTMWN17PRM191HV',
  'HostId': 'nks6ThZ2+ePTyM55HmElw2SI330/Vi1nZAzo2RLfPhp513mggLbiqbev+9nrrOhGB1tMgai6j3s=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'nks6ThZ2+ePTyM55HmElw2SI330/Vi1nZAzo2RLfPhp513mggLbiqbev+9nrrOhGB1tMgai6j3s=',
   'x-amz-request-id': 'DKTMWN17PRM191HV',
   'date': 'Thu, 12 Dec 2024 18:26:28 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [20]:
print("===============================Knowledge base==============================")
knowledge_base_metadata.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)

===============================Knowledge base==============================
======== Vector Index, collection and associated policies deleted =========
======== Knowledge base and data source deleted =========
An error occurred (NoSuchBucket) when calling the ListObjects operation: The specified bucket does not exist
No intermediate bucket found
Found role AmazonBedrockExecutionRoleForKnowledgeBase_2090331
======Attached policies with role AmazonBedrockExecutionRoleForKnowledgeBase_2090331========
 [{'PolicyName': 'AmazonBedrockCloudWatchPolicyForKnowledgeBase_2090331', 'PolicyArn': 'arn:aws:iam::533267284022:policy/AmazonBedrockCloudWatchPolicyForKnowledgeBase_2090331'}, {'PolicyName': 'AmazonBedrockOSSPolicyForKnowledgeBase_2090331', 'PolicyArn': 'arn:aws:iam::533267284022:policy/AmazonBedrockOSSPolicyForKnowledgeBase_2090331'}, {'PolicyName': 'AmazonBedrockS3PolicyForKnowledgeBase_2090331', 'PolicyArn': 'arn:aws:iam::533267284022:policy/AmazonBedrockS3PolicyForKnowledgeBase_2090331'